In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS,DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [4]:
!pip install -r /Users/kailashdejesushornig/Documents/GitHub/CV_projects/requirements.txt

  Using cached shap-0.46.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached kaggle-1.6.14-py3-none-any.whl
ERROR: Could not find a version that satisfies the requirement PyPDFLoader (from versions: none)
ERROR: No matching distribution found for PyPDFLoader


# A simple script for RAG experiments 

Most code taken from: https://github.com/Tech-Watt/RAG-Application-Using-OpenAI-Model/blob/main/main.ipynb 

In [2]:
load_dotenv()
# More models: https://platform.openai.com/docs/models/gpt-4o
model = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'),model='gpt-4o-mini' )
model.invoke('what is the meaning of life?') #Where we pass the question

AIMessage(content='The question "What is the meaning of life?" has been a central philosophical inquiry for centuries and can be approached from various perspectives:\n\n1. **Philosophical Perspective**: Different philosophical traditions offer varied answers. Existentialists, like Jean-Paul Sartre, suggest that life has no inherent meaning and it is up to each individual to create their own purpose. In contrast, some religious philosophies, like those in many forms of Christianity, Islam, and Hinduism, propose that life\'s meaning is derived from a higher power or divine plan.\n\n2. **Scientific Perspective**: From a biological standpoint, the meaning of life could be seen as survival and reproduction. Evolutionary theory suggests that life exists to perpetuate itself through natural selection.\n\n3. **Psychological Perspective**: Psychologists such as Viktor Frankl argue that finding meaning is crucial for mental health and well-being. According to Frankl\'s logotherapy, meaning can 

In [72]:
# Creating a chain to get the pure reponse without clutter 
parser = StrOutputParser()
chain = model|parser # So called chaining
chain.invoke('what is Chalmers in Sweden?') #Less verbose

"Chalmers University of Technology, commonly known simply as Chalmers, is a prestigious technical university located in Gothenburg, Sweden. It was founded in 1829 and is named after its benefactor, William Chalmers, a director of the Swedish East India Company who bequeathed part of his fortune to establish the school. \n\nChalmers is renowned for its research and education in technology, natural sciences, architecture, maritime management, and other engineering disciplines. The university emphasizes innovation, sustainability, and collaboration with industry. It offers a range of undergraduate, master's, and doctoral programs and is known for its strong international presence and partnerships.\n\nThe university is also involved in various cutting-edge research projects and has a number of specialized research centers and labs. It operates two campuses in Gothenburg: Johanneberg and Lindholmen."

# Loading pdfs

In [73]:
#example with only one file
# file_loader = PyPDFLoader('/Users/kailashdejesushornig/Documents/GitHub/CV_projects/data/governing_docs_chalmers/Alkohol- och drogpolicy.PDF')
# page = file_loader.load_and_split()
# len(page) # number of pages read

7

In [5]:
#My extended version to load entire directories
import os
from tqdm import tqdm
import fitz
# from PyPDFLoader import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the directory containing the PDFs
pdf_directory = '/Users/kailashdejesushornig/Documents/GitHub/CV_projects/data/governing_docs_chalmers'

# Initialize the PDF loader
file_loader = PyPDFLoader()

# Initialize the text splitter
spliter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50) # experiment with parameters

# Get a list of PDF files
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# Process each PDF file with a tqdm progress bar
for filename in tqdm(pdf_files, desc="Processing PDFs"):
    pdf_path = os.path.join(pdf_directory, filename)
    
    # Load and split the PDF
    doc = file_loader.load_and_split(pdf_path)
    
    # Chunk each split document
    for page in doc:
        pages = spliter.split_documents(page)
        
    # Look at the first chunk of the first page
    print(pages[0])
#print(len(pages))


ModuleNotFoundError: No module named 'PyPDFLoader'

In [7]:
!pip install fitz

  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata (816 bytes)
  Using cached configobj-5.0.8-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached configparser-7.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached nibabel-5.2.1-py3-none-any.whl.metadata (8.8 kB)
  Using cached nipype-1.8.6-py3-none-any.whl.metadata (6.6 kB)
  Using cached pyxnat-1.6.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached prov-2.0.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached rdflib-7.0.0-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 12.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached etelemetry-0.3.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached looseversion-1.3.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached lxml-5.2.2-cp312-cp312-macosx_1

In [74]:
#The doc is still to large, chunk it
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50) #experiment here? eg. 10 -> destroys the understanding, 50+ works

# Todo: write a script that optimizes the chunk sizes (obs this has been solved before)

pages = spliter.split_documents(page)
pages[0] #look at first

Document(page_content='STYRDOKUMENT: Riktlinje för inköp och upphandling vid Chalmers tekniska högskola . Dnr C 20 2 1 -\n1529 . Beslut av CPO Tommy Bothin , 20 2 1 - 10 - 08 .Dokumentets metadata:\nBeslut av:', metadata={'source': 'Riktlinje för inköp och upphandling vid Chalmers tekniska högskola 211008.pdf', 'page': 0})

In [75]:
# FAISS from facebook 
vector_storage = FAISS.from_documents(pages,OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

# todo: change to chroma

In [77]:
question_template = """
You're a smart bot that answers questions based on the context given to you only.
You don't make things up, this is important.
context:{context}
question:{question}

"""

prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'
              ))


your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context: Here is the context to use
question: Answer this question based on the context




In [80]:
#Passing to vectorDB (fetch context) and to LLM in parallell (ask based on fetched context)
result = RunnableParallel(context= retriever,question = RunnablePassthrough()) # explain this again
chain = result |prompt | model | parser # new chain
#question based on wrong context
chain.invoke('What is a LLM?')

'The provided context does not contain any information about what "LLM" or "LLMs" refers to. Therefore, based on the given context, I cannot provide an answer to your question.'

In [81]:
#question based on right context
chain.invoke('Vem får ingå avtal för Chalmers räkning?')

'Enligt den givna kontexten får endast den som har delegation ingå avtal för Chalmers räkning.'

In [83]:
#to get the source page we use the retreiver
retriever.invoke('Vem får ingå avtal för Chalmers räkning?')

#First one correct: Document(page_content='Chalmers, 201 8 - 06 - 25 , C201 8 - 0 262 i dess senaste version) får företräda Chalmers. Endast den som\nhar delegation kan ingå avtal för Chalmers räkning.', metadata={'source': 'Riktlinje för inköp och upphandling vid Chalmers tekniska högskola 211008.pdf', 'page': 1}),

[Document(page_content='Chalmers, 201 8 - 06 - 25 , C201 8 - 0 262 i dess senaste version) får företräda Chalmers. Endast den som\nhar delegation kan ingå avtal för Chalmers räkning.', metadata={'source': 'Riktlinje för inköp och upphandling vid Chalmers tekniska högskola 211008.pdf', 'page': 1}),
 Document(page_content='3 Grundläggande förutsättningar\nChalmers är en upphandlande enhet och omfattas av lagstiftningen avseende offentlig upphandling.\nVid all upphandling ska särskilt följande principer beaktas:', metadata={'source': 'Riktlinje för inköp och upphandling vid Chalmers tekniska högskola 211008.pdf', 'page': 0}),
 Document(page_content='– Styrdokument vid Chalmers , den 31 oktober 201 6, C 2016 - 1759 .\n2 Mål\nMål et är att visa att Chalmers gör goda affärer .\n3 Grundläggande förutsättningar', metadata={'source': 'Riktlinje för inköp och upphandling vid Chalmers tekniska högskola 211008.pdf', 'page': 0}),
 Document(page_content='Institutionerna ansvarar utifrån sin